In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
import pandas as pd

# Load the CSV file
fdf = pd.read_csv('fdf.csv')

In [3]:
pd.set_option('display.max_columns', None)
fdf.sample(2)

,act_wt,age,age_re,b4_flow,b4_win,colour,country of origin,course,dam's sire,date,declarhorse_wt,dist,draw,draw_counts,gear_list,going,horse_id,horse_name,horse_no,hv_rain,hv_temp,import type,jockey,last rating,n_raceindex,no. of 1-2-3-starts*,p_odds,p_odds_allo,p_trainer_counts,pace_sp,pace_sp_shifted,place,placing,placing_shifted,prize,q_odds,qp_odds,quinella,quinella place,race,raceclass,rc,rc_counts,result_counts,rtg.,rtg_race,same sire,same_sire_score,season stakes*,sex,sire,sire_score,st_rain,st_temp,start ofseason rating,time_per_distance,total stakes*,track,track_counts,train_gear,trainer,trot_sum,type_counts,venue_counts,w_odds,w_odds_allo,win,year
386,126,7.0,7.0,49.0,28.0,Bay,AUS,B+2,Ice Point,2024-01-07,1115,1600.0,1,{},{B},G,2018_C517,ALL FOR ST PAUL'S,4.0,0.0,20.0,PPG,M F Poon,93.0,23/24_324,9-7-3-46,7.9,4130807.0,"{""R.B."": 33}",6.018125,6.145556,39555603.0,13,11,3120000.0,38.0,14.0,61569309.0,53053464.0,10.0,C2,ST,13.0,{},93.0,100-80,"{2018_C102,2023_J117,2020_E061,2019_D375}",111.0,3853200.0,Gelding,Magnus,61.0,0.0,19.4,93.0,{},14079105.0,turf,"{""SmT"": 21, ""AWT"": 6, ""TroR"": 6, ""Turf"": 1}","{""H"": 33, ""B"": 1}",F C Lor,39.0,"{""Trotting"": 27, ""Gallop"": 6}","{""Sha Tin"": 6, ""Conghua"": 27, ""Happy Valley"": 1}",39.0,1053366.0,49795487.0,2024
8519,132,4.0,2.0,124.0,2.6,Chestnut,AUS,A,Show A Heart,2022-12-11,1315,1200.0,9,{},NaN,G,2021_G346,MIGHTY STRIDE,2.0,0.0,17.3,PPG,C Lemaire,59.0,22/23_234,3-2-0-8,1.2,17957399.0,"{""R.B."": 8, ""A.T."": 11}",5.793333,5.722500,26119853.0,1,1,1080000.0,3.8,2.0,35774456.0,35625418.0,2.0,C4,ST,14.0,{},59.0,60-40,"{2022_H059,2021_G088,2023_J223,2022_H184,2022_...",56.0,111600.0,Gelding,Rich Enuff,23.0,0.0,16.0,59.0,{},3085050.0,turf,"{""SmT"": 12, ""TroR"": 4, ""Turf"": 2, ""AWT"": 2}","{""H"": 6}",P F Yiu,27.0,"{""Swimming"": 14, ""Trotting"": 16, ""Gallop"": 3}","{""Sha Tin"": 5, ""Conghua"": 29}",2.3,11298623.0,31499192.0,2022


In [4]:
def top_3(row):
    if float(row['p_odds']) < 1.6:
        return 0
    elif int(row['placing']) <= 3:
        return 1
    else:
        return 0

def top_1(row):
  if float(row['w_odds']) < 3:
     return 0
  elif int(row['placing'])==1:
    return 1
  else:
    return 0

In [5]:
from pandas import json_normalize
import pandas as pd

# # Prepare the data for encoding
# gear_list_series = df['gear_list'].apply(lambda x: ','.join(x) if isinstance(x, list) else x)

def explode_cols(df, cols_to_convert):

    for col in cols_to_convert:
        # Check if the column exists in the dataframe
        if col in df.columns:
            # Normalize the data for the current column
            expanded_data = json_normalize(df[col])
            
            # Replace NaN with 0
            expanded_data.fillna(0, inplace=True)

            # Rename columns of expanded_data to avoid name conflicts when joining back
            expanded_data.columns = [f"{col}_{sub_col}" for sub_col in expanded_data.columns]

            # Drop the original column and join the expanded data back to the main dataframe
            df = df.drop(col, axis=1).join(expanded_data)

    return df


# Perprocess train_gear cols
def combine_and_clean_columns_train(df):
    # 1. Drop columns ending with '-'
    df = df.drop(columns=[col for col in df.columns if col.endswith('-')])

    # 2. Combine columns with trailing numbers into their base columns
    base_cols = [col.rsplit('_', 1)[0] for col in df.columns if col[-1].isdigit() and '_' in col]
    for base_col in set(base_cols):
        related_cols = [col for col in df.columns if col.startswith(base_col) and 
                        (col[-1].isdigit()) and col != base_col]
        if related_cols:
            if base_col not in df.columns:
                df[base_col] = 0
            for rc in related_cols:
                df[base_col] += df[rc]
                df.drop(columns=rc, inplace=True)

    return df

def process_train_gear(df):
    
    # Normalize the 'train_gear' column to separate columns
    expanded_data = json_normalize(df['train_gear'])

    # Replace NaN with 0
    expanded_data.fillna(0, inplace=True)

    # Prefix the columns for clarity
    expanded_data.columns = [f"train_gear_{col}" for col in expanded_data.columns]

    # Clean and combine columns
    cleaned_train_gear_df = combine_and_clean_columns_train(expanded_data)

    # Drop the original 'train_gear' column and merge the new cleaned dataframe

    df = pd.concat([df, cleaned_train_gear_df], axis=1)
    df = df.drop('train_gear', axis=1)
    
    return df

In [6]:
from sklearn.preprocessing import OneHotEncoder

def handle_gear_string(gear_string):
    if gear_string == '{""}':
        return []
    elif gear_string and isinstance(gear_string, str):
        # Remove curly braces and split by comma
        gears = gear_string.strip('{}').split(',')
        return gears
    else:
        return []

        
def combine_and_clean_columns_race(df):
    
    cols = df.columns.tolist()
    
    # Create a set to keep track of base columns to combine
    base_cols = set([col if '-' not in col and not col[-1].isdigit() else col[:-2] if col[-1].isdigit() and '-' in col else col[:-1] for col in cols])
    
    # Remove columns with '-'
    for col in cols:
        if '-' in col:
            df.drop(col, axis=1, inplace=True)
    
    # Combine related columns
    for base_col in base_cols:
        related_cols = [col for col in df.columns if col.startswith(base_col) and (col == base_col or col[len(base_col)].isdigit())]
        if len(related_cols) > 1:
            df[base_col] = df[related_cols].sum(axis=1)
            # Drop all but the base_col
            for rc in related_cols:
                if rc != base_col:
                    df.drop(rc, axis=1, inplace=True)
                
    return df

def label_encoded_info(df):

    # Preprocess the 'gear_list' column using handle_gear_string function
    df['gear_list'] = df['gear_list'].apply(handle_gear_string)

    # Prepare the data for encoding gear_list
    gear_encoded = df['gear_list'].apply(lambda x: ','.join(x) if isinstance(x, (list, set)) else '').str.get_dummies(sep=',')
    
    print(gear_encoded.columns)  # Printing the columns before cleaning
    gear_encoded = combine_and_clean_columns_race(gear_encoded)
    print(gear_encoded.columns)  # Printing the columns after cleaning
    
    # Drop the original 'gear_list' and 'same sire' columns and merge the new encoded dataframes
    df = df.drop(['gear_list'], axis=1)
    df = pd.concat([df, gear_encoded], axis=1)
        
    return df

def process_same_sire_column(orig_df):
    df = orig_df.copy()
    # Define a function to clean each entry in the 'same sire' column
    def clean_entry(entry):
        # Convert to string
        entry_str = str(entry)
        # Remove unwanted characters
        cleaned = entry_str.replace("[", "").replace("]", "").replace('"', "").strip()
        # Split by comma, then strip leading/trailing spaces for each entry, and rejoin
        cleaned_list = [item.strip() for item in cleaned.split(',')]
        cleaned = ','.join(cleaned_list)
        if not cleaned: # If empty after cleaning
            return None
        return cleaned

    # Clean the 'same sire' column
    df['cleaned_same_sire'] = df['same sire'].apply(clean_entry)

    # # Drop rows where 'cleaned_same_sire' is None
    # df = df[df['cleaned_same_sire'].notna()]

    # # Get dummies
    # same_sire_encoded = df['cleaned_same_sire'].str.get_dummies(sep=',')

    # # Drop the original 'same sire' and temporary columns, and merge the new encoded dataframe
    # df = df.drop(['same sire', 'cleaned_same_sire'], axis=1)
    # df = pd.concat([df, same_sire_encoded], axis=1)

    return df

In [7]:
from sklearn.preprocessing import LabelEncoder

def label_encode_columns(df, columns):

    le = LabelEncoder()
    
    for col in columns:
        df[col + '_encoded'] = le.fit_transform(df[col])
    
    return df

In [8]:
def drop_col(df):
    # df = df.dropna(subset=['race_pla'])
    cols_to_drop = ['age', 'b4_flow', 'b4_win', 
                    'date',   'gear_list',  'horse_name',  
                    'n_raceindex', 'p_trainer_counts',  'same sire',   
                     'year' ]

    # Use the 'drop' method and specify axis=1 to drop columns
    df = df.drop(columns=cols_to_drop, errors='ignore')

    return df

In [9]:
# Process the dataframe
def process_dataframe(orig_df):
    
    """
    Process the dataframe by applying a series of transformations
    including exploding columns, combining and cleaning columns, and processing train gear data.
    """
    df = orig_df.copy()

    df['top3'] = df.apply(top_3, axis=1)
    df['top1'] = df.apply(top_1, axis=1)
    # Apply the transformations
    cols_to_convert = ['draw_counts','p_trainer_counts', 'track_counts', 'result_counts','time_per_distance', 
                       'type_counts', 'venue_counts']
    
    df = explode_cols(df, cols_to_convert)

    df = process_train_gear(df)
    
    df = label_encoded_info(df)
    
#     df = process_same_sire_column(df)
    
    # Drop cols
    df = drop_col(df)
    
    float_covert = ['act_wt','age_re','declarhorse_wt', 'draw', 'hv_rain','hv_temp', 
                       'last rating',
                     'p_odds', 'pace_sp_shifted', 'placing_shifted', 
                   'place','prize', 'quinella', 'quinella place', 'rc_counts',
                    'rtg.','same_sire_score', 'season stakes*', 'sire_score', 
                    'st_temp', 'start ofseason rating',
                    'st_rain','total stakes*', 'w_odds', 'win']
    df[float_covert] = df[float_covert].astype(float)
    
    return df

In [10]:
label_df = process_dataframe(fdf)

Index(['B', 'B-', 'B1', 'B2', 'BO', 'BO-', 'BO1', 'BO2', 'CP', 'CP-', 'CP1',
       'CP2', 'E', 'E-', 'E1', 'E2', 'H', 'H-', 'H1', 'H2', 'P', 'P-', 'P1',
       'P2', 'PC', 'PC-', 'PC1', 'PC2', 'SB', 'SB-', 'SB1', 'SR', 'SR-', 'SR1',
       'SR2', 'TT', 'TT-', 'TT1', 'TT2', 'V', 'V-', 'V1', 'V2', 'VO', 'VO-',
       'VO1', 'XB', 'XB-', 'XB1', 'XB2'],
      dtype='object')
Index(['B', 'BO', 'CP', 'E', 'H', 'P', 'PC', 'SB', 'SR', 'TT', 'V', 'VO',
       'XB'],
      dtype='object')


In [11]:
sorted_df = label_df.sort_index(axis=1)

In [12]:

# Find columns that have NaN values
nan_columns = sorted_df.columns[sorted_df.isna().any()].tolist()

# Print the column names that contain NaN values
print("Columns with NaN values:", nan_columns)

sorted_df[sorted_df['rtg_race'].isna()].head()


Columns with NaN values: ['horse_no', 'p_odds', 'p_odds_allo', 'q_odds', 'qp_odds', 'rtg_race', 'season stakes*', 'w_odds', 'w_odds_allo']


,B,BO,CP,E,H,P,PC,SB,SR,TT,V,VO,XB,act_wt,age_re,colour,country of origin,course,dam's sire,declarhorse_wt,dist,draw,going,horse_id,horse_no,hv_rain,hv_temp,import type,jockey,last rating,no. of 1-2-3-starts*,p_odds,p_odds_allo,pace_sp,pace_sp_shifted,place,placing,placing_shifted,prize,q_odds,qp_odds,quinella,quinella place,race,raceclass,rc,rc_counts,rtg.,rtg_race,same_sire_score,season stakes*,sex,sire,sire_score,st_rain,st_temp,start ofseason rating,top1,top3,total stakes*,track,trainer,trot_sum,w_odds,w_odds_allo,win
51,0,0,1,0,0,0,0,0,0,0,0,0,0,126.0,5.0,"{Bay,Brown}",AUS,A,Rahy,1230.0,1600.0,3.0,G,2020_E436,2.0,0.05,19.3,PPG,J McDonald,113.0,6-4-2-15,1.2,12435671.0,5.873125,5.895625,18088249.0,1,2.0,13000000.0,3.2,1.6,27186881.0,22913374.0,7.0,G1,ST,8.0,113.0,NaN,285.0,14615250.0,Gelding,Deep Field,53.0,0.0,17.7,113.0,0,0,41052975.0,turf,P F Yiu,100.0,1.9,12940499.0,29802362.0
73,0,0,1,0,0,0,0,0,0,1,0,0,0,126.0,5.0,Bay,AUS,A,Redoute's Choice,1217.0,1600.0,4.0,G,2020_E269,8.0,0.05,19.3,PPG,K Teetan,96.0,6-3-2-30,9.3,1604603.0,5.915625,5.894286,18088249.0,5,6.0,13000000.0,70.0,17.0,27186881.0,22913374.0,7.0,G1,ST,8.0,96.0,NaN,87.0,1035050.0,Gelding,Written Tycoon,33.0,0.0,17.7,96.0,0,0,11070175.0,turf,D A Hayes,46.0,81.0,303543.0,29802362.0
84,0,0,0,0,1,0,0,0,0,1,0,0,0,126.0,5.0,Chestnut,AUS,A,Jeune,1035.0,1600.0,5.0,G,2020_E058,6.0,0.05,19.3,PP,K C Leung,117.0,5-5-5-26,2.1,7106098.0,5.894375,5.916250,18088249.0,3,5.0,13000000.0,7.7,3.2,27186881.0,22913374.0,7.0,G1,ST,8.0,117.0,NaN,79.0,4473500.0,Gelding,Sebring,49.0,0.0,17.7,117.0,0,1,22744200.0,turf,A S Cruz,108.0,12.0,2048912.0,29802362.0
86,0,0,0,0,0,0,0,0,0,0,0,0,0,126.0,4.0,Bay,AUS,A,Tavistock,1127.0,1600.0,1.0,G,2021_G435,3.0,0.05,19.3,PP,C Y Ho,117.0,4-0-1-15,2.2,6783093.0,5.918125,6.106000,18088249.0,6,4.0,13000000.0,9.7,2.7,27186881.0,22913374.0,7.0,G1,ST,8.0,117.0,NaN,92.0,6298000.0,Gelding,Fastnet Rock,16.0,0.0,17.7,117.0,0,0,14733000.0,turf,C Fownes,121.0,11.0,2235177.0,29802362.0
97,0,0,0,0,0,0,0,0,1,0,0,0,0,126.0,5.0,Bay,IRE,A,Fastnet Rock,1160.0,1600.0,6.0,G,2020_E198,4.0,0.05,19.3,PP,A Atzeni,121.0,4-3-2-28,9.9,1507354.0,5.922500,6.123000,18088249.0,7,8.0,13000000.0,83.0,19.0,27186881.0,22913374.0,7.0,G1,ST,8.0,121.0,NaN,-19.0,781650.0,Gelding,Galileo,21.0,0.0,17.7,121.0,0,0,48780298.0,turf,D J Whyte,80.0,70.0,351242.0,29802362.0


In [13]:
# from pycaret.classification import *
import pandas as pd
import numpy as np
import time
# Define features and targets
top1_df = sorted_df.drop(columns=['top3', 'horse_no'])
top3_df = sorted_df.drop(columns=['top1', 'horse_no'])

In [14]:
categorical_features = ['colour','country of origin','course', 'dist', 'draw',
                        "dam's sire", 'going', 'jockey', 'import type', 'raceclass','rc', 'rtg_race', 
                        'sex', 'sire', 'track', 
                        'horse_id', 'trainer']

ignore_features = ['qp_odds','q_odds', 'no. of 1-2-3-starts*', 'pace_sp', 'placing','quinella','quinella place']

ordinal_features =  {'raceclass' : ['c5', 'c4', 'c3', 'c2', 'c1','other', 'g3', 'g1', 'g2']}

In [15]:
pd.set_option('display.max_columns', None)
sorted_df['raceclass'].unique()

array(['C4', 'C2', 'C3', 'C5', 'G1', 'G3', 'C1', 'G2', 'Other'],
      dtype=object)

## Setup()

In [16]:
pd.set_option('display.max_columns', None)
sorted_df.sample(5)

,B,BO,CP,E,H,P,PC,SB,SR,TT,V,VO,XB,act_wt,age_re,colour,country of origin,course,dam's sire,declarhorse_wt,dist,draw,going,horse_id,horse_no,hv_rain,hv_temp,import type,jockey,last rating,no. of 1-2-3-starts*,p_odds,p_odds_allo,pace_sp,pace_sp_shifted,place,placing,placing_shifted,prize,q_odds,qp_odds,quinella,quinella place,race,raceclass,rc,rc_counts,rtg.,rtg_race,same_sire_score,season stakes*,sex,sire,sire_score,st_rain,st_temp,start ofseason rating,top1,top3,total stakes*,track,trainer,trot_sum,w_odds,w_odds_allo,win
8615,1,0,0,0,0,0,0,0,0,1,0,0,0,132.0,3.0,Grey,AUS,awt,Danerich,1183.0,1200.0,6.0,G,2020_E320,5.0,0.00,22.1,PP,L Hewitson,37.0,2-1-2-25,1.4,13181415.0,5.733333,5.815000,22368462.0,1,3.0,810000.0,5.7,2.5,30467930.0,29831564.0,1.0,C5,ST,12.0,37.0,40-0,30.0,NaN,Gelding,The Factor,0.0,0.0,20.1,37.0,1,0,1605250.0,awt,D A Hayes,51.0,3.7,5187513.0,23265212.0
4512,0,0,0,0,0,0,0,0,0,1,0,0,0,122.0,2.0,Bay,AUS,A,Fastnet Rock,1229.0,1200.0,1.0,G,2022_H170,7.0,0.05,28.8,PP,K C Leung,64.0,1-2-2-9,2.2,9884890.0,5.832500,5.823333,26359708.0,6,4.0,1725000.0,11.0,4.1,44296778.0,38540731.0,10.0,C3,ST,12.0,64.0,80-60,0.0,1639950.0,Gelding,Dream Ahead,14.0,0.0,28.4,64.0,0,0,1803825.0,turf,T P Yung,38.0,11.0,3041607.0,40554762.0
12859,0,0,0,0,0,0,0,0,0,1,0,0,0,125.0,2.0,Bay,AUS,B+2,Bel Esprit,1155.0,1200.0,11.0,G,2021_G003,10.0,0.00,17.2,PPG,K Teetan,52.0,2-0-1-18,2.3,8774986.0,5.857500,5.866667,24463598.0,5,5.0,1040000.0,8.5,4.2,32216823.0,30833924.0,2.0,C4,ST,14.0,52.0,60-40,0.0,222750.0,Gelding,Akeed Mofeed,-1.0,0.0,16.8,52.0,0,0,1634550.0,turf,W Y So,54.0,8.4,2706519.0,27557288.0
10392,0,0,0,0,1,0,0,0,1,0,0,0,0,131.0,3.0,Bay,NZ,A,Volksraad,1024.0,1650.0,4.0,G,2020_E296,3.0,0.50,28.7,PPG,R Maia,36.0,1-1-1-27,2.5,6166735.0,6.149091,6.206111,18687075.0,6,11.0,780000.0,15.0,5.4,24518862.0,24100744.0,3.0,C5,HV,11.0,36.0,40-0,30.0,48125.0,Gelding,Rip Van Winkle,-12.0,6.5,28.5,36.0,0,0,866175.0,turf,K W Lui,55.0,8.9,1798462.0,19401589.0
3827,0,0,0,0,0,0,0,0,0,1,1,0,0,132.0,5.0,"{Bay,Brown}",AUS,A,Fastnet Rock,1103.0,1600.0,3.0,G,2019_D472,4.0,13.00,29.5,PPG,M Chadwick,36.0,2-1-3-35,4.8,3531411.0,6.089375,5.987143,20546392.0,9,7.0,810000.0,36.0,13.0,29082467.0,27404246.0,2.0,C5,ST,14.0,36.0,40-0,43.0,NaN,Gelding,More Than Ready,-7.0,6.5,29.1,36.0,0,0,2172000.0,turf,F C Lor,19.0,16.0,1075081.0,20850048.0


In [17]:
from category_encoders import CatBoostEncoder
from pycaret.classification import *

encoder = CatBoostEncoder()

t3_model = setup(data = top3_df, target = 'top3', experiment_name = 'Top3 predict', train_size=0.7,
                 # fix_imbalance = True,fix_imbalance_method = 'SMOTEENN',
                 remove_outliers = True,
                 imputation_type = 'iterative',
                 numeric_imputation = 'mean',
                 categorical_imputation = 'mode',
                 # numeric_iterative_imputer = 'rf',
                 # categorical_iterative_imputer = 'rf',
                 #use_gpu = True,
                 # fold = 20,
                 remove_multicollinearity = True,
                 multicollinearity_threshold = 0.95,
                 # feature_selection = True, 
                 # feature_selection_estimator = 'gbc', feature_selection_method = 'classic',
                 # n_features_to_select = 0.6,
                 # pca = True,
                 # pca_method = 'linear',
                 # pca_components = 0.95,
                 # max_encoding_ohe = 10,
                 categorical_features = categorical_features,
                 encoding_method = encoder,
                 ignore_features = ignore_features,
                 ordinal_features = ordinal_features,
                 # numeric_features = numeric_features,
                 transformation = True,
                 data_split_stratify = True, 
                 normalize = True,
                 session_id=42)

[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004578 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7110
[LightGBM] [Info] Number of data points in the train set: 11320, number of used features: 53
[LightGBM] [Info] Start training from score 5.990018
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, 

,Description,Value
0,Session id,42
1,Target,top3
2,Target type,Binary
3,Original data shape,"(16186, 64)"
4,Transformed data shape,"(15619, 135)"
5,Transformed train set shape,"(10763, 135)"
6,Transformed test set shape,"(4856, 135)"
7,Ignore features,7
8,Ordinal features,1
9,Numeric features,39


In [18]:
pd.set_option('display.max_columns', None)
get_config('dataset_transformed').describe()

,B,BO,CP,E,H,P,PC,SB,SR,TT,V,VO,XB,act_wt,age_re,colour_Brown,colour_Grey,colour_Bay,colour_Chestnut,colour_BayBrown,colour_Dark BayBrown,colour_BrownGrey,colour_Dark Bay,colour_Roan,colour_Black,colour_BayGrey,country of origin_AUS,country of origin_NZ,country of origin_FR,country of origin_IRE,country of origin_USA,country of origin_CHI,country of origin_GB,country of origin_SAF,country of origin_GER,country of origin_JPN,country of origin_BRZ,country of origin_ARG,country of origin_ITY,course_C+3,course_C,course_A+3,course_B,course_A,course_awt,course_B+2,dam's sire,declarhorse_wt,dist_1200.0,dist_1400.0,dist_1650.0,dist_1600.0,dist_2000.0,dist_1000.0,dist_1800.0,dist_2400.0,dist_2200.0,draw_3.0,draw_11.0,draw_9.0,draw_2.0,draw_10.0,draw_14.0,draw_12.0,draw_1.0,draw_7.0,draw_4.0,draw_8.0,draw_6.0,draw_5.0,draw_13.0,going_G,going_GF,going_WS,going_GY,going_S,going_FT,going_Y,going_H,going_YS,going_WF,horse_id,hv_rain,import type_ISG,import type_PPG,import type_PP,jockey,last rating,p_odds_allo,pace_sp_shifted,place,placing_shifted,prize,race,raceclass_-1.0,rc,rc_counts,rtg_race_80-60,rtg_race_60-40,rtg_race_100-80,rtg_race_40-0,rtg_race_105-80,rtg_race_100-75,rtg_race_60-35,rtg_race_115-90,rtg_race_100-70,rtg_race_80-55,rtg_race_90-65,rtg_race_85-60,rtg_race_110-85,rtg_race_90-70,rtg_race_95-75,rtg_race_95+,rtg_race_90+,rtg_race_105-75,rtg_race_110-80,same_sire_score,season stakes*,sex_Gelding,sex_Colt,sex_Horse,sex_Mare,sex_Filly,sire,sire_score,st_rain,st_temp,total stakes*,track,trainer,trot_sum,w_odds,w_odds_allo,win,top3
count,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.0,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000,15619.000000
mean,-0.005468,0.007569,0.004318,0.001462,0.003624,-0.006992,0.004887,0.000795,0.001985,-0.005974,-0.001874,-0.004239,0.011036,-0.007166,0.001429,-0.007433,0.010552,-0.003288,0.006857,0.003121,-0.002692,-0.001700,-0.001691,-0.009141,0.008726,-0.009944,-0.004121,-0.003110,0.006545,-0.003224,0.009984,-0.002792,0.009208,0.000090,0.001078,-0.005565,0.008129,0.003935,-0.005995,-0.002467,0.001587,0.003818,0.001723,-0.003559,0.005157,-0.003982,-0.040260,-0.003702,-0.004679,-0.000771,0.002416,0.007575,0.006497,0.000471,-0.006783,-0.001702,0.005902,0.010387,0.000269,0.002841,0.002170,-0.003237,-0.006659,-0.003123,0.005359,-0.004061,0.001292,-0.001888,-0.004599,-0.005671,0.007036,-0.011833,0

In [22]:
# from category_encoders import CatBoostEncoder
# encoder = CatBoostEncoder()

# t1_model = setup(data = top1_df, target = 'top1', experiment_name = 'Top1 predict', train_size=0.7,
#                  fix_imbalance = True,fix_imbalance_method = 'SMOTE',
#                  remove_outliers = True,
#                  # imputation_type = 'iterative',
#                 # numeric_imputation = 'mode',
#                 #  categorical_imputation = 'mode',
#                  # numeric_iterative_imputer = 'rf',
#                  # categorical_iterative_imputer = 'rf',
#                  #use_gpu = True,
#                  # fold = 20,
#                  remove_multicollinearity = True,
#                  multicollinearity_threshold = 0.9,
#                  # feature_selection = True, 
#                  # feature_selection_estimator = 'rf', feature_selection_method = 'classic',
#                  # n_features_to_select = 0.3,
#                  # pca = True,
#                  max_encoding_ohe = 15,
#                  categorical_features = categorical_features,
#                  encoding_method = encoder,
#                  ignore_features = ignore_features,
#                  # numeric_features = numeric_features,
#                  transformation = True,
#                  data_split_stratify = True, 
#                  normalize = True,
#                  session_id=42)

## EDA

In [19]:
# eda(display_format = 'bokeh')
pd.set_option('display.max_columns', None)
selected_data = get_config('dataset_transformed')
selected_data.sample(5)

,B,BO,CP,E,H,P,PC,SB,SR,TT,V,VO,XB,act_wt,age_re,colour_Brown,colour_Grey,colour_Bay,colour_Chestnut,colour_BayBrown,colour_Dark BayBrown,colour_BrownGrey,colour_Dark Bay,colour_Roan,colour_Black,colour_BayGrey,country of origin_AUS,country of origin_NZ,country of origin_FR,country of origin_IRE,country of origin_USA,country of origin_CHI,country of origin_GB,country of origin_SAF,country of origin_GER,country of origin_JPN,country of origin_BRZ,country of origin_ARG,country of origin_ITY,course_C+3,course_C,course_A+3,course_B,course_A,course_awt,course_B+2,dam's sire,declarhorse_wt,dist_1200.0,dist_1400.0,dist_1650.0,dist_1600.0,dist_2000.0,dist_1000.0,dist_1800.0,dist_2400.0,dist_2200.0,draw_3.0,draw_11.0,draw_9.0,draw_2.0,draw_10.0,draw_14.0,draw_12.0,draw_1.0,draw_7.0,draw_4.0,draw_8.0,draw_6.0,draw_5.0,draw_13.0,going_G,going_GF,going_WS,going_GY,going_S,going_FT,going_Y,going_H,going_YS,going_WF,horse_id,hv_rain,import type_ISG,import type_PPG,import type_PP,jockey,last rating,p_odds_allo,pace_sp_shifted,place,placing_shifted,prize,race,raceclass_-1.0,rc,rc_counts,rtg_race_80-60,rtg_race_60-40,rtg_race_100-80,rtg_race_40-0,rtg_race_105-80,rtg_race_100-75,rtg_race_60-35,rtg_race_115-90,rtg_race_100-70,rtg_race_80-55,rtg_race_90-65,rtg_race_85-60,rtg_race_110-85,rtg_race_90-70,rtg_race_95-75,rtg_race_95+,rtg_race_90+,rtg_race_105-75,rtg_race_110-80,same_sire_score,season stakes*,sex_Gelding,sex_Colt,sex_Horse,sex_Mare,sex_Filly,sire,sire_score,st_rain,st_temp,total stakes*,track,trainer,trot_sum,w_odds,w_odds_allo,win,top3
11511,-0.642813,-0.066227,-0.291325,-0.10663,-0.409089,-0.161322,-0.130416,-0.023617,-0.198497,1.042237,-0.22463,-0.013633,-0.369021,1.262882,0.089819,-0.398663,5.329832,-1.339182,-0.409862,-0.180061,-0.039774,-0.060305,-0.054608,-0.039774,-0.028929,-0.031985,-1.045151,-0.570663,6.722584,-0.369508,-0.120482,-0.050149,-0.212973,-0.108837,-0.08708,-0.045257,-0.086536,-0.034775,-0.019282,-0.497153,-0.485496,-0.242333,2.244959,-0.558213,-0.293945,-0.280487,-1.285379,-0.063818,1.213018,-0.466405,-0.394898,-0.280102,-0.136192,-0.360190,-0.26435,-0.048251,-0.091829,-0.294690,-0.273487,-0.28145,-0.303904,-0.285657,-0.166404,-0.266556,-0.293198,-0.297289,3.282638,-0.30171,-0.299505,-0.305179,-0.168457,0.555635,-0.460044,-0.08922,-0.165221,-0.082065,-0.045257,-0.122842,-0.043147,-0.027273,-0.023617,-1.846030,-0.526853,-0.230502,0.931279,-0.840545,-0.351372,1.065962,1.752389,0.260497,1.137540,-0.365255,0.586720,1.195790,0.0,-1.266555,-0.384538,1.446964,-0.922616,-0.234938,-0.295806,-0.111429,-0.140035,-0.094367,-0.039774,-0.08922,-0.048251,-0.033409,-0.128562,-0.073607,-0.048251,-0.063334,-0.036089,-0.030495,-0.025511,-0.009639,1.214202,0.994478,0.064796,-0.036089,-0.044215,-0.023617,-0.019282,-0.913652,0.881567,-0.635676,0.384668,1.187188,0.293945,0.212348,-0.432277,-1.295509,1.514223,1.043573,0
7311,-0.642813,-0.066227,-0.291325,-0.10663,-0.409089,-0.161322,-0.130416,-0.023617,-0.198497,1.042237,-0.22463,-0.013633,-0.369021,-1.472789,0.089819,-0.398663,-0.187623,0.746725,-0.409862,-0.180061,-0.039774,-0.060305,-0.054608,-0.039774,-0.028929,-0.031985,-1.045151,1.752349,-0.148752,-0.369508,-0.120482,-0.050149,-0.212973,-0.108837,-0.08708,-0.045257,-0.086536,-0.034775,-0.019282,-0.497153,-0.485496,-0.242333,-0.445442,-0.558213,-0.293945,3.565225,-0.409379,-0.946160,1.213018,-0.466405,-0.394898,-0.280102,-0.136192,-0.360190,-0.26435,-0.048251,-0.091829,-0.294690,-0.273487,-0.28145,-0.303904,-0.285657,-0.166404,-0.266556,-0.293198,-0.297289,-0.304633,-0.30171,-0.299505,-0.305179,5.936249,0.555635,-0.460044,-0.08922,-0.165221,-0.082065,-0.045257,-0.122842,-0.043147,-0.027273,-0.023617,-0.220403,-0.526853,-0.230502,0.931279,-0.840545,-0.910387,0.155601,-1.400287,-0.139494,1.722082,0.637133,0.810991,1.525163,0.0,0.789543,1.259207,1.446964,-0.922616,-0.234938,-0.295806,-0.111429,-0.140035,-0.094367,-0.039774,-0.08922,-0.048251,-0.033409,-0.128562,-0.073607,-0.048251,-0.063334,-0.036089,-0.030495,-0.025511,-0.009639,-0.89

In [24]:
# from autoviz.AutoViz_Class import AutoViz_Class

# AV = AutoViz_Class()

# report = AV.AutoViz(
#     filename = '',
#     dfte=selected_data,
#     depVar='top3',
#     verbose=0,
#     lowess=False,
#     chart_format='svg',
#     max_rows_analyzed=1500,
#     max_cols_analyzed=30
# )


## Custom Metrics

In [20]:
def top_3_metrics(y, y_pred):
    tp = np.where((y_pred==1) & (y==1), 60, 0)
    fp = np.where((y_pred==1) & (y==0), -30, 0)
    tn = np.where((y_pred==0) & (y==0), 0, 0)
    fn = np.where((y_pred==0) & (y==1), -10, 0)
    return np.sum([tp, fp, tn, fn])

In [21]:
get_metrics(reset = True)
add_metric('T3_metric', 'T3_metric', top_3_metrics)

Name                                                         T3_metric
Display Name                                                 T3_metric
Score Function       <pycaret.internal.metrics.EncodedDecodedLabels...
Scorer               make_scorer(top_3_metrics, response_method='pr...
Target                                                            pred
Args                                                                {}
Greater is Better                                                 True
Multiclass                                                        True
Custom                                                            True
Name: T3_metric, dtype: object

In [22]:
%%time
# compare baseline models
top3 = compare_models(
                    n_select = 3,
                    sort = 'T3_metric',
                    probability_threshold = 0.4,
                    exclude = 'et'
                    )

tuned_top3 = [tune_model(i) for i in top3]
blender = blend_models(tuned_top3, choose_better = True, probability_threshold = 0.2)
stacker = stack_models(tuned_top3, choose_better = True, probability_threshold = 0.2)
best_top3_model = automl(optimize = 'Prec.', turbo = False)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,14:53:00
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Extra Trees Classifier


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,T3_metric,TT (Sec)
lr,Logistic Regression,0.7462,0.6595,0.1994,0.3534,0.2522,0.1150,0.1224,0.0000,82.9930
knn,K Neighbors Classifier,0.6076,0.5402,0.3952,0.2481,0.3047,0.0510,0.0537,0.0000,39.2950
nb,Naive Bayes,0.2032,0.4742,0.8793,0.1950,0.3191,-0.0025,-0.0176,0.0000,42.3350
dt,Decision Tree Classifier,0.6070,0.5045,0.3231,0.2664,0.2919,0.1013,0.1022,0.0000,80.3270
svm,SVM - Linear Kernel,0.7492,0.5865,0.0940,0.2748,0.1382,0.0330,0.0394,0.0000,80.6990
ridge,Ridge Classifier,0.7776,0.6619,0.0272,0.3522,0.0500,0.0203,0.0424,0.0000,81.6690
rf,Random Forest Classifier,0.7539,0.7225,0.2890,0.4082,0.3381,0.1925,0.1969,0.0000,81.8630
qda,Quadratic Discriminant Analysis,0.4229,0.5084,0.6449,0.2201,0.3219,0.0043,0.0050,0.0000,81.4250
ada,Ada Boost Classifier,0.2958,0.7377,0.9988,0.2360,0.3818,0.0457,0.1516,0.0000,81.7010
gbc,Gradient Boosting Classifier,0.7627,0.7495,0.3315,0.4393,0.3773,0.2346,0.2383,0.0000,81.5250


Processing:   0%|          | 0/63 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [27]:
available_plots = ['confusion_matrix', 'feature']
    
for i in available_plots:
    print('_________________________________________  ',i,'  ___________________________________________',sep=' ')
    plot_model(best_top3_model, plot=i)
    print('\n\n')

_________________________________________   confusion_matrix   ___________________________________________


NameError: name 'plot_model' is not defined

In [41]:
save_model(top3[1], 'Test_top3_model_112023')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/tmp/joblib),
          steps=[('iterative_imputer',
                  TransformerWrapper(exclude=None, include=None,
                                     transformer=IterativeImputer(add_indicator=False,
                                                                  cat_estimator=LGBMClassifier(boosting_type='gbdt',
                                                                                               class_weight=None,
                                                                                               colsample_bytree=1.0,
                                                                                               importance_type='split',
                                                                                               learning_rate=0.1,
                                                                                               max_depth=-1,
                                                                   

## Catboost model

In [ ]:
catboost_model = create_model('gbc',
                        probability_threshold = 0.35
                       )
tuned_rf = tune_model(catboost_model, 
                     optimize = 'AUC',
                     search_library = 'optuna',
                     n_iter = 20,
                     # search_algorithm = 'optuna'
                     early_stopping = True
                     )

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,T3_metric
Fold,,,,,,,,
0,0.7488,0.7487,0.4305,0.4286,0.4295,0.2684,0.2684,650.0000
1,0.7704,0.7700,0.4574,0.4766,0.4668,0.3206,0.3207,1550.0000
2,0.7448,0.7446,0.3857,0.4135,0.3991,0.2373,0.2376,130.0000
3,0.7823,0.7522,0.4170,0.5054,0.4570,0.3224,0.3247,1550.0000
4,0.7724,0.7751,0.4395,0.4804,0.4590,0.3153,0.3158,1450.0000
5,0.7310,0.7295,0.4215,0.3950,0.4078,0.2341,0.2343,30.0000
6,0.7860,0.7755,0.4234,0.5137,0.4642,0.3320,0.3345,1690.0000
7,0.7653,0.7570,0.4081,0.4619,0.4333,0.2860,0.2869,960.0000
8,0.7751,0.7917,0.4843,0.4887,0.4865,0.3426,0.3426,1940.0000


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,02:12:18
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Gradient Boosting Classifier


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

[I 2023-11-22 02:12:18,393] Searching the best hyperparameters using 10146 samples...


In [ ]:
available_plots = ['confusion_matrix', 'feature','auc']
    
for i in available_plots:
    print('_________________________________________  ',i,'  ___________________________________________',sep=' ')
    plot_model(tuned_rf, plot=i)
    print('\n\n')

In [54]:
save_model(tuned_rf, 'gbc_top3_model_112023')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/tmp/joblib),
          steps=[('iterative_imputer',
                  TransformerWrapper(exclude=None, include=None,
                                     transformer=IterativeImputer(add_indicator=False,
                                                                  cat_estimator=LGBMClassifier(boosting_type='gbdt',
                                                                                               class_weight=None,
                                                                                               colsample_bytree=1.0,
                                                                                               importance_type='split',
                                                                                               learning_rate=0.1,
                                                                                               max_depth=-1,
                                                                   